In [ ]:
import sys
import gzip
import numpy as np
import glob
import collections
import operator
from pprint import pprint  

Caso esteja a utilizar o google colab, deve seguir as instruções disponíveis no moodle
e correr a célula seguinte

In [ ]:
# Apenas para quem usa o google colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Iscte/TM/data

In [ ]:
ficheiros=glob.glob("../data/pt/tsf.selecionado/*.gz")
print(ficheiros)

In [ ]:
textos = []
for fname in ficheiros:
    f=gzip.open(fname, "rt", encoding="utf-8")
    texto = f.readlines()
    texto = "".join(texto)
    textos.append(texto)
print("Tenho guardados %d documentos"% len(textos))

In [ ]:
print(textos[0])

Agora vamos extrair contagens de palavras (features) a partir dos documentos.

Nesta fase talvez não fosse má ideia fazer algum processamento, por exemplo, para passar a ter apenas palavras em letras minúsculas e evitar ter palavras no vocabulário tais como `«perfeito»` ou `«já`...

In [ ]:
def doc2wc(texto):   
    palavras = texto.lower().strip().split()
    return collections.Counter(palavras)

doc2wc("A capa do livro do Pedro")

In [ ]:
docs = []
for texto in textos:
    contagens = doc2wc(texto)
    docs.append(contagens)
print(docs[0])

Talvez não fosse má ideia fazer algum processamento para, por exemplo:
 * passar a ter apenas palavras em letras minúsculas
 * eliminar tokens tais como `(`, `:`, `«`, etc...
 * evitar ter palavras no vocabulário tais como `«perfeito»` ou `«já`.

Vamos agora ver qual é o vocabulário que está a ser usado ...

In [ ]:
voc = set()
for d in docs:
    for w in d:
        voc.add(w)

In [ ]:
print("O vocabulário tem %d palavras"% len(voc) )
voc

### Este vocabulário é agora usado para representar os nossos documentos

Vamos agora criar um identificador para cada palavra do vocabulário

In [ ]:
def create_index(voc):
    indice = {}
    for i,w in enumerate(sorted(voc)):
        indice[w] = i
    return indice

In [ ]:
indice = create_index(voc)
indice

Agora sim, vamos representar os nossos documentos em vetores numéricos

In [ ]:
docrep = []
for d in docs:
    valores = np.zeros([len(voc)], dtype=int)
    for w in d:
        if w in indice:
            valores[ indice[w] ] = d[w]
    docrep.append ( valores )
docrep

Vamos lá ver qual o documento mais próximo do terceiro

In [ ]:
from scipy.spatial import distance
ref=3
for i in range(len(docrep)):
    print("similarity(%d,%d)=%f"% (ref,i, 1 - distance.cosine(docrep[ref], docrep[i])) )

In [ ]:
print(textos[ref])

In [ ]:
print(textos[6])

## IDF para identificar os termos mais relevantes

In [ ]:
def idf(df, ndocs):
    return np.log(ndocs/df)

In [ ]:
tf = collections.Counter()
df = collections.Counter()
for d in docs:
    for w in d:
        tf[w] += d[w]
        df[w] += 1
        
idfs = {}
for w in tf:
    if tf[w] > 1:
        idfs[w] = idf(df[w], len(docs))

In [ ]:
for w in ["em", "com", "de", "uefa", "marcador", "voto", "cristiano", "tap"]:
    print(w, tf[w], df[w], len(docs), idf(df[w], len(docs)))

In [ ]:
sorted(idfs.items(), key=operator.itemgetter(1), reverse=True)

Agora que sabemos quais as palavras mais importantes, podemos definir novo vocabulário ...

In [ ]:
voc=sorted(idfs, key=idfs.get, reverse=True)[:200]
voc

Podemos agora voltar acima e representar os documentos com este novo vocabulário, definindo novo índice ... 
Talvez chegue à conclusão que o par de documentos mais similar não é o 3 e o 5.